In [2]:
import pandas as pd
import re

movies = pd.read_csv('data/movies_data/ratings_movies.csv')
movies.head()

,Unnamed: 0,userId,movieId,rating,date,title,genres
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [26]:
def get_year_release(title):
    for i in range(len(title) - 5):
        if title[i] == '(' and title[i+5] == ')' and title[i+1: i+5].isdigit():
            return int(title[i+1: i+5])
    
    return 99999
                       
movies['year_release'] = movies['title'].apply(get_year_release)
movies[movies['year_release'] == 99999].shape[0]

18

In [41]:
movies[movies['year_release'] == 1999].groupby('title')['rating'].mean().sort_values()

title
Bloodsport: The Dark Kumite (1999)            0.5
Simon Sez (1999)                              1.0
Chill Factor (1999)                           1.0
Source, The (1999)                            1.0
Trippin' (1999)                               1.0
                                             ... 
Trailer Park Boys (1999)                      5.0
Larry David: Curb Your Enthusiasm (1999)      5.0
Sun Alley (Sonnenallee) (1999)                5.0
George Carlin: You Are All Diseased (1999)    5.0
Five Senses, The (1999)                       5.0
Name: rating, Length: 261, dtype: float64

In [43]:
movies[movies['year_release'] == 2010].groupby('genres')['rating'].mean().sort_values()

genres
Action|Sci-Fi                        1.000000
Action|Adventure|Horror              1.500000
Action|Drama|Fantasy                 1.500000
Crime|Romance                        1.500000
Adventure|Comedy|Fantasy             1.833333
                                       ...   
Crime                                4.750000
Comedy|Musical                       5.000000
Animation|Drama|Fantasy|Mystery      5.000000
Adventure|Children|Comedy|Mystery    5.000000
Animation|Children|Mystery           5.000000
Name: rating, Length: 119, dtype: float64

In [51]:
movies.groupby('userId')['genres'].agg('nunique').sort_values()

userId
214     13
85      13
245     13
494     15
578     15
      ... 
474    395
380    399
448    403
414    482
599    524
Name: genres, Length: 610, dtype: int64

In [56]:
movies.groupby('userId')['rating'].agg(
    ['count', 'mean']
    ).sort_values(
        by=['count', 'mean'],
        ascending=[True, False])

,count,mean
userId,,
53,20,5.000000
595,20,4.200000
189,20,4.100000
569,20,4.000000
278,20,3.875000
...,...,...
274,1346,3.235884
448,1864,2.847371
474,2108,3.398956


In [75]:
req = movies[movies['year_release'] == 2018].groupby(
    'genres')['rating'].agg(['count', 'mean'])
req[req['count'] > 10].sort_values('count', ascending=False).index[0]

'Action|Adventure|Sci-Fi'

In [120]:
movies['year_rating'] = pd.to_datetime(movies['date']).dt.year

pivot = pd.pivot_table(
    data=movies,
    values='rating',
    columns='year_rating',
    index='genres',
    fill_value=0
)

pivot[2018].sort_values(ascending=False)


genres
Comedy|Horror|Mystery                     5.0
Comedy|Fantasy|Horror|Musical|Thriller    5.0
Animation|Children|Mystery                5.0
Comedy|Crime|Horror|Thriller              5.0
Drama|Horror|Mystery|Sci-Fi|Thriller      5.0
                                         ... 
Adventure|Comedy|Crime|Drama|Romance      0.0
Adventure|Comedy|Crime|Mystery            0.0
Comedy|Drama|Sci-Fi|War                   0.0
Comedy|Drama|Sci-Fi|Thriller              0.0
Adventure|Drama|Horror|Thriller           0.0
Name: 2018, Length: 951, dtype: float64